In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [ ]:
# 祖传参数
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "verbose": -1,
         "nthread": 4,
         "random_state": 666}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=666)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()
feature_importance_favorite = pd.DataFrame(df_train_columns, columns = ['feature'])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['label'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=target1.iloc[trn_idx], categorical_feature=categorical_features)
    val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=target1.iloc[val_idx], categorical_feature=categorical_features)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
    feature_importance_favorite[f"importance_fold{fold_}"] = clf.feature_importance()
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = df_train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    predictions += clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

print(roc_auc_score(target1.values, oof))